In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch.nn as nn
import torch.nn.functional as F
import gym
import torch
import argparse
from environment import atari_env
from utils import read_config
from torchvision import transforms
import numpy as np
from torch.optim import Adam
from environment import atari_env
from rnet import RepresentNet, TDClass
torch.manual_seed(1)

parser = argparse.ArgumentParser(description='A3C')
parser.add_argument(
    '--env',
    default='Pong-v0',
    metavar='ENV',
    help='environment to train on (default: Pong-v0)')
parser.add_argument(
    '--env-config',
    default='config.json',
    metavar='EC',
    help='environment to crop and resize info (default: config.json)')
parser.add_argument(
    '--skip-rate',
    type=int,
    default=4,
    metavar='SR',
    help='frame skip rate (default: 4)')
parser.add_argument(
    '--max-episode-length',
    type=int,
    default=10000,
    metavar='M',
    help='maximum length of an episode (default: 10000)')
args = parser.parse_args([])
setup_json = read_config(args.env_config)
env_conf = setup_json["Default"]
for i in setup_json.keys():
    if i in args.env:
        env_conf = setup_json[i]


In [2]:
transform = transforms.ToTensor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

r_net = RepresentNet().to(device)
c_net = TDClass().to(device)
optimizer_r = Adam(r_net.parameters(), lr=1e-4)
optimizer_c = Adam(c_net.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

batch_size = 8
trace = []
td_class = [(0,1),(1,2),(2,3),(3,5),(5,7),(7,9)]

env = atari_env(args.env, env_conf, args)  # gym.make("Pong-v0")
# 先累积轨迹
s = env.reset()
print(s.shape)
for i in range(50000):
    s = torch.from_numpy(s).float().to(device)
    trace.append(s)
    s, r, done, _ = env.step(env.action_space.sample())
    if done:
        print("reset")
        s = env.reset()
        for i in range(20):  # 前面20帧显示不完整
            s, r, done, _ = env.step(env.action_space.sample())
    if i%1000==0:
        print("accumulate", i)

print("begin train")
# 训练
for epoch in range(5000000):
    range_c = np.random.randint(0, len(td_class))
    TD = np.random.randint(td_class[range_c][0], td_class[range_c][1])
    begin = np.random.randint(0, len(trace)-TD-batch_size)
    former = torch.stack(trace[begin:begin+batch_size], dim=0).to(device)
    latter = torch.stack(trace[begin+TD:begin+TD+batch_size], dim=0).to(device)
    target = torch.zeros(batch_size, dtype=torch.long).to(device) + range_c

    rep_f, rep_l = r_net(former), r_net(latter)
    output = c_net(rep_f, rep_l, epoch%30==0)
    loss = loss_fn(output, target)

    optimizer_r.zero_grad()
    optimizer_c.zero_grad()
    loss.backward()
    if epoch%500==0:
#   print(c_net.l_2.weight.grad.max())
        print("grad", r_net.conv1[0].weight.grad.mean().item())
#     nn.utils.clip_grad_norm_(r_net.parameters(), 20.)
#     nn.utils.clip_grad_norm_(c_net.parameters(), 20.)
    optimizer_r.step()
    optimizer_c.step()
    if epoch%500==0:
        print("range {} TD {} loss {:.3f}".format(range_c, TD, loss.item()))
    if epoch%10000==0:
        print(output)
        # 测试
        all_num = 0
        accu_num = 0
        for _ in range(50):
            range_c = np.random.randint(0, len(td_class))
            TD = np.random.randint(td_class[range_c][0], td_class[range_c][1])
            begin = np.random.randint(0, len(trace)-TD-batch_size)
            former = torch.stack(trace[begin:begin+batch_size], dim=0).to(device)
            latter = torch.stack(trace[begin+TD:begin+TD+batch_size], dim=0).to(device)
            target = torch.zeros(batch_size, dtype=torch.long).to(device) + range_c
            rep_f, rep_l = r_net(former), r_net(latter)
            output = c_net(rep_f, rep_l, False)
            accu_num += (output.max(dim=1)[1]==target).sum().item()
            all_num += batch_size
        print("accu_rate {:.4f}".format(accu_num/all_num))

(1, 80, 80)
accumulate 0
accumulate 1000
reset
accumulate 2000
reset
accumulate 3000
reset
accumulate 4000
reset
accumulate 5000
accumulate 6000
reset
accumulate 7000
reset
accumulate 8000
reset
accumulate 9000
reset
accumulate 10000
reset
accumulate 11000
accumulate 12000
reset
accumulate 13000
reset
accumulate 14000
reset
accumulate 15000
reset
accumulate 16000
accumulate 17000
reset
accumulate 18000
reset
accumulate 19000
reset
accumulate 20000
reset
accumulate 21000
reset
accumulate 22000
accumulate 23000
reset
accumulate 24000
reset
accumulate 25000
reset
accumulate 26000
reset
accumulate 27000
reset
accumulate 28000
accumulate 29000
reset
accumulate 30000
reset
accumulate 31000
reset
accumulate 32000
reset
accumulate 33000
reset
accumulate 34000
reset
accumulate 35000
accumulate 36000
reset
accumulate 37000
reset
accumulate 38000
reset
accumulate 39000
reset
accumulate 40000
accumulate 41000
reset
accumulate 42000
reset
accumulate 43000
reset
accumulate 44000
reset
accumulate 450

grad 0.010084555484354496
range 3 TD 3 loss 1.715
grad -1.7865570043795742e-05
range 0 TD 0 loss 1.044
grad -0.053876686841249466
range 1 TD 1 loss 1.404
grad -4.0950193236355403e-10
range 0 TD 0 loss 1.044
grad -0.05274391174316406
range 0 TD 0 loss 1.103
grad 0.0005766956601291895
range 3 TD 4 loss 1.810
grad 0.010853364132344723
range 4 TD 6 loss 1.486
grad -0.00038861858774907887
range 2 TD 2 loss 1.084
grad -0.025729691609740257
range 0 TD 0 loss 1.155
grad 0.011436701752245426
range 5 TD 7 loss 1.193
grad 0.015925776213407516
range 1 TD 1 loss 1.111
grad 0.0007589988526888192
range 4 TD 6 loss 1.662
grad -0.006774645298719406
range 2 TD 2 loss 1.399
grad 0.00028770731296390295
range 4 TD 5 loss 1.522
grad 1.0046086572401691e-05
range 3 TD 3 loss 1.044
grad 0.005030754953622818
range 3 TD 4 loss 1.638
grad 2.411154014225758e-07
range 5 TD 7 loss 1.044
grad -3.4091500455968315e-11
range 0 TD 0 loss 1.044
grad 0.004171980079263449
range 4 TD 6 loss 1.205
grad -0.0006780943367630243


grad 0.0002470241452101618
range 2 TD 2 loss 1.045
grad 0.004708219785243273
range 4 TD 5 loss 1.302
grad 0.0007273222436197102
range 2 TD 2 loss 1.418
grad 0.061537113040685654
range 1 TD 1 loss 1.305
grad 0.0011536234524101019
range 4 TD 6 loss 1.175
grad 0.0005180853186175227
range 3 TD 3 loss 1.046
grad -0.005735444836318493
range 2 TD 2 loss 1.367
grad -9.030095005080341e-10
range 0 TD 0 loss 1.044
grad -0.000355670228600502
range 5 TD 8 loss 1.293
grad -0.00123221252579242
range 5 TD 7 loss 1.485
grad 0.006963716354221106
range 3 TD 3 loss 1.381
grad -0.0005130321369506419
range 0 TD 0 loss 1.419
grad 2.6718123990576714e-05
range 2 TD 2 loss 1.294
grad -1.4713737783722536e-08
range 4 TD 5 loss 1.044
grad -3.637401269074303e-09
range 0 TD 0 loss 1.044
grad 0.00032781041227281094
range 2 TD 2 loss 1.044
grad 1.243540168616164e-06
range 3 TD 3 loss 1.044
grad -0.019039347767829895
range 4 TD 6 loss 1.727
grad -4.744939792640512e-13
range 0 TD 0 loss 1.044
grad 0.005102221854031086
r

grad -0.00012509168300312012
range 2 TD 2 loss 1.544
grad -0.00014222069876268506
range 2 TD 2 loss 1.234
grad -1.8427658371215433e-17
range 1 TD 1 loss 1.044
tensor([[ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0')
accu_rate 0.7400
grad 5.085877546662232e-06
range 4 TD 6 loss 1.044
grad -8.357551115104242e-26
range 0 TD 0 loss 1.044
grad 0.00015660151257179677
range 0 TD 0 loss 1.168
grad 0.0013058128533884883
range 4 TD 6 loss 1.045
grad 1.642711566773869e-08
range 3 TD 3 loss 1.044
grad -3.7620015973516274e-06
range 3 

grad 0.006059864070266485
range 2 TD 2 loss 1.832
grad 0.02156233973801136
range 2 TD 2 loss 1.356
grad 0.00015439461276400834
range 4 TD 6 loss 1.668
grad 1.351418982303145e-32
range 5 TD 8 loss 1.044
grad -0.0024997955188155174
range 2 TD 2 loss 1.288
grad -7.433233986375853e-05
range 3 TD 3 loss 1.419
grad 0.007998659275472164
range 1 TD 1 loss 1.177
grad -0.0054643298499286175
range 5 TD 7 loss 2.034
grad -7.439140148335355e-08
range 0 TD 0 loss 1.044
grad -8.544919838045119e-20
range 5 TD 7 loss 1.044
grad 0.014210746623575687
range 2 TD 2 loss 1.410
grad -0.007701395079493523
range 1 TD 1 loss 1.125
tensor([[ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.2892,  0.2267,  0.0000,  0.4540,  0.0301],
        [ 0.0000,  0.9999,  0.0000,  0.0000,  0.0001,  0.0000],
        [ 0.0016,  0.9984,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0001,  0.9999,  0.0000,  0.0000,  0.0000,  0.0000],
   

KeyboardInterrupt: 

In [1]:
torch.save(r_net.cpu(), "pre_models/r_net_5wsam.pkl")
torch.save(c_net.cpu(), "pre_models/c_net_5wsam.pkl")

NameError: name 'torch' is not defined